# "Pasma energetyczne"

In [56]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as anim
from matplotlib.animation import FuncAnimation, HTMLWriter
from IPython.core.display import HTML

In [57]:
def gauss(x, a):
    return (2 * a / np.pi)**(1/4) * np.exp(-a * x**2)

In [58]:
dt = 0.004 * 10**(-15)
dx = 0.1 * 10**(-9)
hbar = 1.054 * 10**(-34)
m = 0.91 * 10**(-30)

In [59]:
class moving_Psi:
    ra = (hbar/(2*m)) * (dt/(dx**2))
    a = 0.4
    b = 2.5 * np.pi
    Re = []
    Im = []
    Im_r = []
    Im_f = []
    P = []
    X = []
    V = []
    def initialise(safe):
        x = -25
        while(x <= 25):
            moving_Psi.Re.append(gauss((x + 6), moving_Psi.a) * np.cos(moving_Psi.b * (x + 6)))
            moving_Psi.Im.append(gauss((x + 6), moving_Psi.a) * np.sin(moving_Psi.b * (x + 6)))
            moving_Psi.Im_r.append(0.0)
            moving_Psi.Im_f.append(0.0)
            moving_Psi.V.append(0.0)
            moving_Psi.P.append((gauss((x + 6), moving_Psi.a) * np.cos(moving_Psi.b * (x + 6)))**2 + (gauss((x + 6), moving_Psi.a) * np.sin(moving_Psi.b * (x + 6)))**2)
            moving_Psi.X.append(x)
            x += 0.025
    def update(safe, n):
        Re = moving_Psi.Re
        Im = moving_Psi.Im
        for k in range(0, n):
            for i in range(1, len(moving_Psi.Re) - 1):
                moving_Psi.Re[i] = Re[i] - moving_Psi.ra * (Im[i + 1] - 2 * Im[i] + Im[i - 1]) + dt/hbar * moving_Psi.V[i] * Im[i]
                moving_Psi.Im[i] = Im[i] + moving_Psi.ra * (Re[i + 1] - 2 * Re[i] + Re[i - 1]) - dt/hbar * moving_Psi.V[i] * Re[i]
                moving_Psi.Im_r[i] = Im[i]
                moving_Psi.Im_f[i] = moving_Psi.Im[i] + moving_Psi.ra * (moving_Psi.Re[i + 1] - 2 * moving_Psi.Re[i] + moving_Psi.Re[i - 1]) - dt/hbar * moving_Psi.V[i] * Re[i]
                moving_Psi.P[i] = moving_Psi.Re[i]**2 + moving_Psi.Im_r[i] * moving_Psi.Im_f[i]
    def set_potential(safe, V):
        for i in range(0, len(moving_Psi.V)):
            moving_Psi.V[i] = V[i] 
    def compute_energy(safe):
        A = hbar**2 / (2 * m * dx**2)
        ke = 0 + 0.j
        pe = 0 + 0.j
        psi = []
        for i in range(0, len(moving_Psi.Re)):
            psi.append(moving_Psi.Re[i] + moving_Psi.Im[i] * 1.j)
            pe += psi[i] * psi[i].conjugate() * moving_Psi.V[i]
        for i in range(1, len(moving_Psi.Re) - 1):
            Lap = psi[i + 1] - 2 * psi[i] + psi[i - 1]
            ke += Lap * psi[i].conjugate() * (moving_Psi.X[i] - moving_Psi.X[i - 1])
        KE = -A * np.real(ke)/(1.6 * 10**(-19))
        PE = np.real(pe)
        return (KE + PE)

In [60]:
# Definicja potencjału
V = []
v = []
x = -25
k = 0
while(x <= 25):
    if((x >= (-18.02 + k) and x <= (-17.98 + k))):
        V.append(-0.01 * 1.6 * 10**(-19))
        k += 1
        v.append(-0.2)
    else:
        V.append(0.0)
        v.append(0.0)
    x += 0.025
     
    
psi = moving_Psi()
psi.initialise()
psi.set_potential(V)      
psi.compute_energy()  

0.14754208228863006

In [ ]:
fig = plt.figure(figsize = (20,8))
line4, = plt.plot([], [],'C4')
line1, = plt.plot([], [],'C0')
line2, = plt.plot([], [],'C1')
line3, = plt.plot([], [],'C2')

plt.grid(linestyle = '--',linewidth = 1)
plt.ylim(-0.75, 0.75)
plt.xlim(-15, 15)
plt.xlabel('$x$')
plt.ylabel('$\Psi(x,t)$')

plt.close()

def frame(i):
    line4.set_data(psi.X, v)
    line1.set_data(psi.X, psi.Re)
    line2.set_data(psi.X, psi.Im)
    line3.set_data(psi.X, psi.P)    
    if(i > 0):
        psi.update(400)
    
    return line1, line2, line3



animation = FuncAnimation(fig, frame, frames = 300, interval = 20)

file = r"grate.html"
writervideo = anim.HTMLWriter(fps=100) 
animation.save(file, writer=writervideo)

#HTML(animation.to_jshtml())